# Some Typical Financial Calculations

# Load Packages

The Roots and Distributions packages have been used in the previous notebooks, but StatsBase not. It contains methods for estimating means, variances, ranks, correlations etc. See https://github.com/JuliaStats/StatsBase.jl. 

In [1]:
using StatsBase, Roots, Distributions

include("printmat.jl")            #function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# CAPM and Return Autocorrelation

The CAPM regression is

$R_{it}^{e}  =\alpha_{i}+\beta_{i}R_{mt}^{e}+\varepsilon_{it}$,

where $R_{mt}^{e}$ is the market excess return. Theory says that $\alpha=0$, which is easily tested.

### Load Data

In [3]:
xx  = readdlm("Data/MyData.csv",',',header=true)        #monthly return data
x   = xx[1]
ym  = x[:,1]                 #yearmonth
Rme = x[:,2]                 #market excess return
Rf  = x[:,3]                 #interest rate
R   = x[:,4]                 #return small growth stocks
Re  = R - Rf                 #excess returns
T   = size(Rme,1)

y  = round.(Int,ym/100)                 #year
m  = round.(Int,ym - y*100)             #month
dN = Date.(y,m)                         #Date, useful for the plots (below)
printmat(dN[1:4])

1979-01-01
1979-02-01
1979-03-01
1979-04-01



### CAPM regression

In [4]:
x    = [ones(T) Rme]             #regressors
y    = copy(Re)                  #to get standard OLS notation
b    = x\y                       #OLS
u    = y - x*b                   #residuals
covb = inv(x'x)*var(u)           #cov(b), see any textbook
stdb = sqrt.(diag(covb))         #std(b)
R2a  = 1 - var(u)/var(y)

println("     coeff      std     t-stat")
printmat([b stdb b./stdb])
printlnPs("R2: ",R2a)
printlnPs("no. of observations: ",T)

     coeff      std     t-stat
    -0.504     0.304    -1.656
     1.341     0.066    20.427

R2:      0.519
no. of observations:        388


## Return Autocorrelations

That is, the correlation of $R_{t}^{e}$ and $R_{t-s}^{e}$. 

(It is easily shown that the t-stat of an autocorrelation is $\sqrt{T}$ times the autocorrelation.)

In [5]:
plags = 1:5
xCorr = autocor(Re,plags)         #using the StatsBase package

println("Autocorrelations of the excess returns in Re")
println("\n     lag   autocorrr    t-stat")
printmat([plags xCorr sqrt(T)*xCorr])

Autocorrelations of the excess returns in Re

     lag   autocorrr    t-stat
     1.000     0.216     4.253
     2.000     0.002     0.046
     3.000    -0.018    -0.359
     4.000    -0.065    -1.289
     5.000    -0.027    -0.536



# Value at Risk

The next cell first constructs an simple estimate of $\sigma_t^2$ as a backward looking moving average (the RiskMetrics approach)

$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) (R_{t-1} -\mu)^2$,
where $\mu$ is the average return (for all data).

Then, we calculate the 95% VaR by assuming a $N(\mu,\sigma_t^2)$ distribution

$\textrm{VaR}_{t} = - (\mu-1.64\sigma_t)$.

If the model is correct, then the loss should not exceed $\textrm{VaR}_{t}$ in more than 5% of the months.

In [6]:
μ = mean(Rme)

λ = 0.95                         #weight on old volatility
σ² = fill(var(Rme),T)                 #RiskMetrics approach to estimate variance     
for t = 2:T
  σ²[t] = λ*σ²[t-1] + (1-λ)*(Rme[t-1]-μ)^2    
end

VaR95 = -(μ - 1.64*sqrt.(σ²))      #VaR at 95% level

println("let's plot it")

let's plot it


In [8]:
xTicksLoc = Dates.value.([Date(1980);Date(1990);Date(2000);Date(2010)])
xTicksLab = ["1980";"1990";"2000";"2010"]               #crude way of getting tick marks right

plot1 = plot(dN,VaR95,color=:blue,legend=false,xticks=(xTicksLoc,xTicksLab))
plot!(ylims=(0,11))
title!("1-month Value at Risk (95%) for US equity market")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-month Value at Risk (95%) for US equity market 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.6753,158.308 46.0851,160.66 47.3585,161.982 48.7683,160.3 50.1327,166.146 51.5425,169.655 52.9068,172.147 54.3167,177.776 55.7265,175.317 57.0908,180.357 
 58.5006,162.929 59.865,162.259 61.2748,167.676 62.6846,165.637 64.0035,170.865 65.4133,124.865 66.7776,128.869 68.1874,130.277 69.5518,135.303 70.9616,133.226 
 72.3714,139.522 73.7358,145.307 75.1456,151.563 76.5099,136.458 77.9197,135.606 79.3295,133.964 80.6029,140.561 82.0127,144.906 83.3771,149.099 84.7869,155.274 
 86.1512,158.855 87.5611,163.502 88.9709,153.418 90.3352,141.292 91.745,143.027 93.1094,147.137 94.5192,148.449 95.929,150.282 97.2024,144.6 98.6122,149.123 
 99.9765,153.488 101.386,154.031 102.751,155.777 104.161,157.979 105.57,134.18 106.935,140.692 108.344,118.488 109.709,121.673 111.119,128.577 112.528,133.189 
 113.802,138.975 115.212,144.128 116.576,140.605 117.986,147.038 119.35,151.596 120.76,152.18 122.17,158.034 123.534,164.008 124.944,164.828 126.308,169.84 
 127.718,173.833 129.128,177.267 130.447,174.456 131.857,180.031 133.221,185.041 134.631,176.707 135.995,181.922 137.405,183.492 138.815,159.892 140.179,165.255 
 141.589,170.3 142.953,174.249 144.363,179.44 145.773,168.694 147.046,174.338 148.456,179.323 149.82,184.031 151.23,183.465 152.595,188.712 154.004,193.413 
 155.414,197.616 156.779,193.605 158.188,195.308 159.553,189.605 160.963,191.737 162.372,196.864 163.646,189.479 165.056,188.977 166.42,192.987 167.83,192.833 
 169.194,197.913 170.604,186.331 172.014,181.818 173.378,163.152 174.788,164.687 176.152,170.431 177.562,171.949 178.972,138.623 180.245,141.405 181.655,147.368 
 183.019,151.583 184.429,157.676 185.793,160.613 187.203,163.305 188.613,167.139 189.977,170.004 191.387,45.0606 192.752,41.2062 194.161,43.2525 195.571,49.6689 
 196.89,55.1136 198.3,62.2608 199.664,70.6769 201.074,78.6406 202.438,83.1397 203.848,90.2921 205.258,94.4807 206.622,100.679 208.032,108.054 209.397,113.452 
 210.806,120.387 212.216,120.088 213.49,125.045 214.899,131.679 216.264,135.122 217.674,140.021 219.038,145.608 220.448,141.014 221.858,147.235 223.222,152.964 
 224.632,154.173 225.996,160.195 227.406,166.022 228.816,152.828 230.089,158.923 231.499,164.501 232.863,165.393 234.273,154.739 235.637,160.046 237.047,164.569 
 238.457,140.319 239.821,135.501 241.231,140.397 242.595,139.205 244.005,144.866 245.415,147.338 246.689,142.324 248.098,147.8 249.463,153.874 250.872,157.461 
 252.237,155.236 253.647,157.691 255.056,162.946 256.421,167.453 257.831,173.035 259.195,171.95 260.605,150.708 262.015,156.572 263.333,162.541 264.743,165.246 
 266.108,171.004 267.517,176.649 268.882,179.662 270.292,182.148 271.701,184.794 273.066,190.061 274.476,195.218 275.84,196.853 277.25,201.586 278.66,206.41 
 279.933,211.148 281.343,214.785 282.707,215.093 284.117,217.927 285.481,222.364 286.891,226.394 288.301,226.843 289.665,230.833 291.075,234.579 292.439,235.777 
 293.849,239.269 295.259,240.923 296.532,240.292 297.942,231.704 299.307,235.827 300.716,239.849 302.081,237.899 303.491,239.836 304.9,239.107 306.265,239.648 
 307.674,243.477 309.039,237.771 310.449,241.723 311.858,245.138 313.132,245.02 314.542,247.602 315.906,250.351 317.316,251.636 318.68,253.289 320.09,252.552 
 321.5,256.14 322.864,256.368 324.274,257.52 325.638,255.89 327.048,259.314 328.458,261.171 329.777,264.345 331.187,267.639 332.551,269.682 333.961,271.369 
 335.325,272.664 336.735,254.696 338.145,255.841 339.509,250.798 340.919,254.384 342.283,243.83 343.693,245.492 345.103,241.125 346.376,244.479 347.786,235.147 
 349.15,234.884 350.56,224.061 351.925,223.761 353.334,211.582 354.744,208.432 356.109,204.94

# Options

## Black-Scholes Option Price

Let $S$ be the the current spot price of an asset and $y$ be the interest rate.

The Black-Scholes formula for a European call option (on that asset) with strike price $K$ and time to expiration $m$ is

$C  =S\Phi\left(  d_{1}\right)  -e^{-ym}K\Phi\left(d_{2}\right)$, where

$d_{1} =\frac{\ln(S/K)+(y+\sigma^{2}/2)m}{\sigma\sqrt{m}} \ \text{ and } \ d_{2}=d_{1}-\sigma\sqrt{m}$ 

and where $\Phi(d)$ denotes the probability of $x\leq d$ when $x$ has an $N(0,1)$ distribution. All variables except the volatility ($\sigma$) are directly observable. 

In [9]:
function Φ(x)
    #Calculates Pr(z<=x) for N(0,1) variable z
    Pr = cdf(Normal(0,1),x)         #Distributions package
  return Pr
end

function OptionBlackSPs(S,K,m,y,σ)
#Calculates Black-Scholes european call option price
  d1 = ( log.(S./K) + (y+1/2*σ.^2)*m ) ./ (σ*sqrt(m))
  d2 = d1 - σ*sqrt(m)
  c  = S.*Φ(d1) - exp(-y*m).*K.*Φ(d2)
  return c
end

OptionBlackSPs (generic function with 1 method)

In [10]:
σ = 0.4
c1 = OptionBlackSPs(10,10,0.5,0.1,σ)
printlnPs("\n","call price according to Black-Scholes: ",c1)

K = linspace(7,13,51)
c = OptionBlackSPs(10,K,0.5,0.1,σ)

println("let's plot it")


call price according to Black-Scholes:      1.358
let's plot it


In [11]:
plot1 = plot(K,c,color=:red,legend=false)
title!("Black-Scholes call option price (for different strike prices)")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Black-Scholes call option price (for different strike prices) 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.0857,31.4961 48.9303,42.6796 59.7749,53.7251 70.6194,64.6249 81.464,75.3715 92.3086,85.9577 103.153,96.3764 113.998,106.621 124.842,116.686 135.687,126.565 
 146.532,136.252 157.376,145.744 168.221,155.035 179.065,164.122 189.91,173.001 200.754,181.67 211.599,190.126 222.444,198.367 233.288,206.392 244.133,214.2 
 254.977,221.791 265.822,229.164 276.667,236.321 287.511,243.262 298.356,249.989 309.2,256.502 320.045,262.805 330.889,268.899 341.734,274.788 352.579,280.474 
 363.423,285.96 374.268,291.25 385.112,296.348 395.957,301.257 406.802,305.981 417.646,310.525 428.491,314.893 439.335,319.089 450.18,323.117 461.025,326.983 
 471.869,330.691 482.714,334.244 493.558,337.649 504.403,340.909 515.247,344.029 526.092,347.014 536.937,349.868 547.781,352.596 558.626,355.202 569.47,357.69 
 580.315,360.065 
 "/>

# Implied Volatility

is the $\sigma$ value that makes the Black-Scholes equation give the same option price as observed on the market. It is often interpreted as the "market uncertainty."

The next cell uses the call option price calculated above as being the "market price." The implied volatility should then be the same as what we used above.

In [12]:
                                #simple (crude) way to solve for implied vol
iv = fzero(σ->OptionBlackSPs(10,10,0.5,0.1,σ)-c1,0.00001,5)

printlnPs("Implied volatility: ",iv,", compare with: $σ")

Implied volatility:      0.400, compare with: 0.4


The next few cells instead some data on options on German government bonds. 

In [13]:
#  LIFFE Bunds option data, trade date April 6, 1994
K = [                        #strike prices; Mx1 vector
      92.00;  94.00;  94.50;  95.00;  95.50;  96.00;  96.50;  97.00;
      97.50;  98.00;  98.50;  99.00;  99.50;  100.0;  100.5;  101.0;
     101.5;  102.0;  102.5;  103.0;  103.5 ];
C = [                        #call prices; Mx1 vector
      5.13;    3.25;    2.83;    2.40;    2.00;    1.64;    1.31;    1.02;
      0.770;   0.570;   0.400;   0.280;   0.190;   0.130;  0.0800;  0.0500;
      0.0400;  0.0300;  0.0200;  0.0100;  0.0100 ];
S = 97.05                #spot price
m = 48/365               #time to expiry in years
y = 0.0                  #Interest rate: LIFFE=>no discounting
N = length(K)

21

In [14]:
iv = fill(NaN,N)
for i = 1:N
  iv[i] = fzero(sigma->OptionBlackSPs(S,K[i],m,y,sigma)-C[i],0.00001,5)
end

println("Strike and iv for data: ")
printmat([K iv])

println("let's plot it")

Strike and iv for data: 
    92.000     0.094
    94.000     0.081
    94.500     0.081
    95.000     0.078
    95.500     0.075
    96.000     0.074
    96.500     0.072
    97.000     0.071
    97.500     0.070
    98.000     0.069
    98.500     0.068
    99.000     0.067
    99.500     0.067
   100.000     0.068
   100.500     0.067
   101.000     0.067
   101.500     0.070
   102.000     0.073
   102.500     0.074
   103.000     0.072
   103.500     0.077

let's plot it


In [15]:
plot1 = plot(K,iv,color=:red,legend=false)
title!("Implied volatility, Bunds options April 6, 1994")
xlabel!("strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 92 
 
 
 94 
 
 
 96 
 
 
 98 
 
 
 100 
 
 
 102 
 
 
 0.07 
 
 
 0.08 
 
 
 0.09 
 
 
 Implied volatility, Bunds options April 6, 1994 
 
 
 strike price

# Mean-variance Frontier

Given a vector of average returns ($\mu$) and a variance-covariance matrix ($\Sigma$), the mean-variance frontier show the lowest possible portfolio uncertainty for a given expected portfolio return. It is thus the solution to a quadratic minimization problem. The cells below will use the explicit (matrix) formulas for this solution, but we often have to resort to numerical methods when there are portfolio restrictions.

It is typically plotted with the portfolio standard deviation on the horizontal axis and the portfolio expected return on the vertical axis.

In [16]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03                       #riskfree return (an interest rate) 

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



The functions in the next cell use the explicit solution to the minimization problem. They assume there are no portfolio restrictions. 

In [17]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [18]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, only risky assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, only risky assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 504.72,294.352 496.741,291.723 488.811,289.094 480.933,286.466 473.111,283.837 465.347,281.209 457.645,278.58 450.008,275.952 442.44,273.323 434.946,270.695 
 427.529,268.066 420.195,265.437 412.948,262.809 405.794,260.18 398.739,257.552 391.788,254.923 384.95,252.295 378.23,249.666 371.636,247.038 365.176,244.409 
 358.859,241.78 352.694,239.152 346.69,236.523 340.858,233.895 335.208,231.266 329.751,228.638 324.5,226.009 319.466,223.381 314.663,220.752 310.102,218.123 
 305.797,215.495 301.763,212.866 298.011,210.238 294.556,207.609 291.41,204.981 288.585,202.352 286.095,199.724 283.948,197.095 282.156,194.466 280.727,191.838 
 279.666,189.209 278.981,186.581 278.674,183.952 278.747,181.324 279.199,178.695 280.028,176.067 281.229,173.438 282.798,170.809 284.727,168.181 287.006,165.552 
 289.626,162.924 292.574,160.295 295.84,157.667 299.41,155.038 303.272,152.41 307.411,149.781 311.815,147.152 316.47,144.524 321.363,141.895 326.482,139.267 
 331.813,136.638 337.344,134.01 343.065,131.381 348.964,128.753 355.031,126.124 361.255,123.495 367.628,120.867 374.14,118.238 380.783,115.61 387.549,112.981 
 394.431,110.353 401.422,107.724 408.516,105.096 415.706,102.467 422.987,99.8385 430.353,97.2099 437.8,94.5814 445.323,91.9528 452.918,89.3243 460.58,86.6957 
 468.306,84.0672 476.093,81.4386 483.937,78.8101 491.835,76.1815 499.784,73.5529 507.781,70.9244 515.824,68.2958 523.911,65.6673 532.04,63.0387 540.207,60.4102 
 548.412,57.7816 556.653,55.1531 564.927,52.5245 573.234,49.8959 581.571,47.2674 589.937,44.6388 598.331,42.0103 606.752,39.3817 615.198,36.7532 623.669,34.1246 
 632.162,31.4961 
 "/>

In [19]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, risky and riskfree assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF, risky and riskfree assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 504.72,294.352 496.741,291.723 488.811,289.094 480.933,286.466 473.111,283.837 465.347,281.209 457.645,278.58 450.008,275.952 442.44,273.323 434.946,270.695 
 427.529,268.066 420.195,265.437 412.948,262.809 405.794,260.18 398.739,257.552 391.788,254.923 384.95,252.295 378.23,249.666 371.636,247.038 365.176,244.409 
 358.859,241.78 352.694,239.152 346.69,236.523 340.858,233.895 335.208,231.266 329.751,228.638 324.5,226.009 319.466,223.381 314.663,220.752 310.102,218.123 
 305.797,215.495 301.763,212.866 298.011,210.238 294.556,207.609 291.41,204.981 288.585,202.352 286.095,199.724 283.948,197.095 282.156,194.466 280.727,191.838 
 279.666,189.209 278.981,186.581 278.674,183.952 278.747,181.324 279.199,178.695 280.028,176.067 281.229,173.438 282.798,170.809 284.727,168.181 287.006,165.552 
 289.626,162.924 292.574,160.295 295.84,157.667 299.41,155.038 303.272,152.41 307.411,149.781 311.815,147.152 316.47,144.524 321.363,141.895 326.482,139.267 
 331.813,136.638 337.344,134.01 343.065,131.381 348.964,128.753 355.031,126.124 361.255,123.495 367.628,120.867 374.14,118.238 380.783,115.61 387.549,112.981 
 394.431,110.353 401.422,107.724 408.516,105.096 415.706,102.467 422.987,99.8385 430.353,97.2099 437.8,94.5814 445.323,91.9528 452.918,89.3243 460.58,86.6957 
 468.306,84.0672 476.093,81.4386 483.937,78.8101 491.835,76.1815 499.784,73.5529 507.781,70.9244 515.824,68.2958 523.911,65.6673 532.04,63.0387 540.207,60.4102 
 548.412,57.7816 556.653,55.1531 564.927,52.5245 573.234,49.8959 581.571,47.2674 589.937,44.6388 598.331,42.0103 606.752,39.3817 615.198,36.7532 623.669,34.1246 
 632.162,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.5598,294.352 59.1535,291.723 63.7472,289.094 68.3409,286.466 72.9346,283.837 77.5283,281.209 82.122,278.58 86.7157,275.952 91.3094,273.323 95.9031,270.695 
 100.497,268.066 105.091,265.437 109.684,262.809 114.278,260.18 118.872,257.552 123.465,254.923 128.059,252.295 132.653,249.666 137.246,247.038 141.84,244.409 
 146.434,241.78 151.028,239.152 155.621,236.523 160.215,233.895 164.809,231.266 169.402,228.638 173.996,226.009 178.59,223.381 183.184,220.752 187.777,218.123 
 192.371,215.495 196.965,212.866 201.558,210.238 206.152,207.609 210.746,204.981 215.339,202.352 219.933,199.724 224.527,197.095 229.121,194.466 233.714,191.838 
 238.308,189.209 242.902,186.581 247.495,183.952 252.089,181.324 256.683,178.695 261.277,176.067 265.87,173.438 270.464,170.809 275.058,168.181 279.651,165.552 
 284.245,162.924 288.839,160.295 293.432,157.667 298.026,155.038 302.62,152.41 307.214,149.781 311.807,147.152 316.401,144.524 320.995,141.895 325.588,139.267 
 330.182,136.638 334.776,134.01 339.37,131.381 343.963,128.753 348.557,126.124 353.151,123.495 357.744,120.867 362.338,118.238 366.932,115.61 371.525,112.981 
 376.119,110.353 380.713,107.724 385.307,105.096 389.9,102.467 394.494,99.8385 399.088,97.2099 403.681,94.5814 408.275,91.9528 412.869,89.3243 417.462,86.6957 
 422.056,84.0672 426.65,81.4386 431.244,78.8101 435.837,76.1815 440.431,73.5529 445.025,70.9244 449.618,68.2958 454.212,65.6673 458.806,63.0387 463.4,60.4102 
 467.993,57.7816 472.587,55.1531 477.181,52.5245 481.774,49.8959 486.368,47.2674 490.962,44.6388 495.555,42.0103 500.149,39.3817 504.743,36.7532 509.337,34.1246 
 513.93,31.4961 
 "/>

# (extra) Mean-variance Frontier without Short Selling

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).
    
The packages MathProgBase and Ipopt are documented at https://github.com/JuliaOpt/MathProgBase.jl and https://github.com/JuliaOpt/Ipopt.jl.

In [20]:
using MathProgBase, Ipopt

In [21]:
function MeanVarNoSSPs(mu,Sigma,mustar)
    K = length(mustar)                #MV with no short-sales, numerical minimization
    N = length(mu)
    vv = find((mustar .>= minimum(mu)) .& (mustar .<= maximum(mu)))  #solve only if feasible
    lb = zeros(N,1)              #w >= 0
    ub = ones(N,1)               #w <= 1
    A  = [ones(1,N);mu']         #1'w=1, mu'w = mustar
    w_p   = fill(NaN,(K,N))
    StdRp = fill(NaN,K)                       #to store results in
    for i = vv
        b  = [1;mustar[i]]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        w_i = Sol.sol
        if Sol.status == :Optimal
          w_p[i,:] = w_i'
          StdRp[i] = sqrt((w_i'*Sigma*w_i)[1])
        end
    end
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [22]:
Std_no_ss, = MeanVarNoSSPs(μ,Σ,mustar)  

plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlims=(0,15),ylims=(0,15))
title!("MVF (portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 MVF (portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 504.72,294.352 496.741,291.723 488.811,289.094 480.933,286.466 473.111,283.837 465.347,281.209 457.645,278.58 450.008,275.952 442.44,273.323 434.946,270.695 
 427.529,268.066 420.195,265.437 412.948,262.809 405.794,260.18 398.739,257.552 391.788,254.923 384.95,252.295 378.23,249.666 371.636,247.038 365.176,244.409 
 358.859,241.78 352.694,239.152 346.69,236.523 340.858,233.895 335.208,231.266 329.751,228.638 324.5,226.009 319.466,223.381 314.663,220.752 310.102,218.123 
 305.797,215.495 301.763,212.866 298.011,210.238 294.556,207.609 291.41,204.981 288.585,202.352 286.095,199.724 283.948,197.095 282.156,194.466 280.727,191.838 
 279.666,189.209 278.981,186.581 278.674,183.952 278.747,181.324 279.199,178.695 280.028,176.067 281.229,173.438 282.798,170.809 284.727,168.181 287.006,165.552 
 289.626,162.924 292.574,160.295 295.84,157.667 299.41,155.038 303.272,152.41 307.411,149.781 311.815,147.152 316.47,144.524 321.363,141.895 326.482,139.267 
 331.813,136.638 337.344,134.01 343.065,131.381 348.964,128.753 355.031,126.124 361.255,123.495 367.628,120.867 374.14,118.238 380.783,115.61 387.549,112.981 
 394.431,110.353 401.422,107.724 408.516,105.096 415.706,102.467 422.987,99.8385 430.353,97.2099 437.8,94.5814 445.323,91.9528 452.918,89.3243 460.58,86.6957 
 468.306,84.0672 476.093,81.4386 483.937,78.8101 491.835,76.1815 499.784,73.5529 507.781,70.9244 515.824,68.2958 523.911,65.6673 532.04,63.0387 540.207,60.4102 
 548.412,57.7816 556.653,55.1531 564.927,52.5245 573.234,49.8959 581.571,47.2674 589.937,44.6388 598.331,42.0103 606.752,39.3817 615.198,36.7532 623.669,34.1246 
 632.162,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 405.063,228.638 393.663,226.009 382.553,223.381 371.764,220.752 361.331,218.123 351.29,215.495 341.683,212.866 332.555,210.238 323.954,207.609 315.933,204.981 
 308.546,202.352 301.851,199.724 295.905,197.095 290.764,194.466 286.482,191.838 283.107,189.209 280.681,186.581 279.233,183.952 278.782,181.324 279.199,178.695 
 280.028,176.067 281.229,173.438 282.798,170.809 284.727,168.181 287.006,165.552 289.626,162.924 292.574,160.295 295.84,157.667 299.41,155.038 303.272,152.41 
 307.411,149.781 311.815,147.152 316.47,144.524 321.363,141.895 326.482,139.267 332.421,136.638 340.671,134.01 351.115,131.381 363.532,128.753 377.694,126.124 
 393.381,123.495 410.393,120.867 428.549,118.238 447.689,115.61 467.679,112.981 488.399,110.353 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

